# Evaluation Malayalam Speech Corpus(MSC) dataset

In [7]:
#| default_exp msc

In [8]:
#| hide
from nbdev.showdoc import *

In [19]:
#| export
import time
from typing import List

import pandas as pd
from datasets import load_dataset, Audio
from faster_whisper import WhisperModel
from jiwer import wer, cer
from transformers import pipeline
from tqdm.notebook import tqdm
from whisper_normalizer.malayalam import MalayalamTextNormalizer

from malayalam_asr_benchmarking.utils import is_target_text_in_range, get_text, data, get_model_size, clear_gpu_memory

## Loading dataset and evaluating model

In [10]:
#| export
def load_malayalam_speech_corpus_dataset():
    dataset = load_dataset(
            "thennal/msc",
            split="train"
    )
    dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
    dataset = dataset.filter(is_target_text_in_range, input_columns=["text"])
    return dataset

In [11]:
#| export
normalizer = MalayalamTextNormalizer()
def evaluate_whisper_model_msc(
        model_name: str, # The model name
        werlist: List[float], # WER List
        cerlist: List[float], # CER list
        modelsizelist: List[str], # model size list
        timelist: List[float], # time(s) list
        bs:int =16, # batch size
)->None:
    whisper_asr = pipeline(
            "automatic-speech-recognition", model=model_name, device=0
        )
    dataset = load_malayalam_speech_corpus_dataset()
    
    predictions = []
    references = []

    start = time.time()
    print("process of calculating predictions")
    for out in tqdm(whisper_asr(data(dataset), batch_size=bs)):
        predictions.append(normalizer(out["text"]))
        references.append(normalizer(out["reference"][0]))
        
    print("completed getting predictions")
    end = time.time()
    print(f"Total time taken: {end - start}")
    timelist.append(end - start)
    
    df = pd.DataFrame({"predictions": predictions, "ground_truth": references})
    df["model_name"] = model_name
    df["wer"] = df.apply(lambda row: wer(normalizer(row["ground_truth"]), normalizer(row["predictions"])), axis=1)
    df["cer"] = df.apply(lambda row: cer(normalizer(row["ground_truth"]), normalizer(row["predictions"])), axis=1)
    df["total_time"] = end-start
    
    rwer = wer(references, predictions)
    rwer = round(100 * rwer, 2)
    df["total_wer"] = rwer
    werlist.append(rwer)
    print(f"The WER of model: {rwer}")

    rcer = cer(references, predictions)
    rcer = round(100 * rcer, 2)
    df["total_cer"] = rcer
    cerlist.append(rcer)
    print(f"The CER of model: {rcer}")
    
    print(f"The model size is: {get_model_size(whisper_asr.model)}")
    modelsizelist.append(get_model_size(whisper_asr.model))
    df["model_size"] = get_model_size(whisper_asr.model)
    
    save_name = model_name.split("/")
    print(save_name)
    df.to_parquet(f"{save_name[0]}_{save_name[1]}_msc.parquet")
    
    clear_gpu_memory()

In [12]:
show_doc(evaluate_whisper_model_msc)

---

[source](https://github.com/kurianbenoy/malayalam_asr_benchmarking/blob/main/malayalam_asr_benchmarking/msc.py#L38){target="_blank" style="float:right; font-size:smaller"}

### evaluate_whisper_model_msc

>      evaluate_whisper_model_msc (model_name:str, werlist:List[float],
>                                  cerlist:List[float], modelsizelist:List[str],
>                                  timelist:List[float], bs:int=16)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| model_name | str |  | The model name |
| werlist | List |  | WER List |
| cerlist | List |  | CER list |
| modelsizelist | List |  | model size list |
| timelist | List |  | time(s) list |
| bs | int | 16 | batch size |
| **Returns** | **None** |  |  |

## Testing with a sample model

In [ ]:
#|eval: false
wer_list = []
cer_list = []
model_size_list = []
time_list = []

In [ ]:
#|eval: false
evaluate_whisper_model_msc("openai/whisper-tiny", wer_list, cer_list, model_size_list, time_list)

Found cached dataset parquet (/home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-30f1618974cdefce.arrow
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e4f860ca9b159c26.arrow


process of calculating predictions


0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


completed getting predictions
Total time taken: 375.53221249580383
The WER of model: 139.63
The CER of model: 177.3
The model size is: 37.76M
['openai', 'whisper-tiny']


In [ ]:
#|eval: false
evaluate_whisper_model_msc("openai/whisper-base", wer_list, cer_list, model_size_list, time_list)

Found cached dataset parquet (/home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-30f1618974cdefce.arrow
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e4f860ca9b159c26.arrow


process of calculating predictions


0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


completed getting predictions
Total time taken: 448.9504859447479
The WER of model: 155.97
The CER of model: 200.05
The model size is: 72.59M
['openai', 'whisper-base']


In [ ]:
#|eval: false
evaluate_whisper_model_msc("openai/whisper-small", wer_list, cer_list, model_size_list, time_list)

Found cached dataset parquet (/home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-30f1618974cdefce.arrow
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e4f860ca9b159c26.arrow


process of calculating predictions


0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


completed getting predictions
Total time taken: 479.73656272888184
The WER of model: 111.57
The CER of model: 123.7
The model size is: 241.73M
['openai', 'whisper-small']


In [ ]:
#|eval: false
evaluate_whisper_model_msc("anuragshas/whisper-large-v2-ml", wer_list, cer_list, model_size_list, time_list, bs=4)

Found cached dataset parquet (/home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-30f1618974cdefce.arrow
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e4f860ca9b159c26.arrow


process of calculating predictions


0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


completed getting predictions
Total time taken: 10467.876322746277
The WER of model: 23.57
The CER of model: 12.33
The model size is: 1.54B
['anuragshas', 'whisper-large-v2-ml']


In [ ]:
#|eval: false
evaluate_whisper_model_msc("DrishtiSharma/whisper-large-v2-malayalam", wer_list, cer_list, model_size_list, time_list, bs=4)

Found cached dataset parquet (/home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-30f1618974cdefce.arrow
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e4f860ca9b159c26.arrow


process of calculating predictions


0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### Faster-whisper models

In [21]:
#| eval: false
dataset = load_malayalam_speech_corpus_dataset()
t = dataset[0]
t

{'speechid': '0I2iIh7OSMdWhvb85tMp',
 'speaker_id': 'oeaNxrE0uxNSfoDpdrCYGBGC7uC3',
 'review_score': 42,
 'transcript': 'അണ്ണാൻ മൂത്താലും മരം കേറ്റം മറക്കുമോ?',
 'category': 'proverb',
 'speaker_gender': 'default',
 'speaker_age': 'default',
 'audio': {'path': None,
  'array': array([-1.16415322e-10, -5.82076609e-11,  0.00000000e+00, ...,
         -3.16558471e-05, -2.90179141e-05, -3.30671537e-05]),
  'sampling_rate': 16000},
 'norm_text': 'അണ ണ ൻ മ ത ത ല മര ക റ റ മറക ക മ '}

In [22]:
#| export
def evaluate_faster_whisper_model_msc(
        model_name: str, # The model name
        werlist: List[float], # WER List
        cerlist: List[float],# CER list
        modelsizelist: List[str], # model size list
        timelist: List[float], # time(s) list
        bs:int =16, # batch size. Default value is 16.
        compute_type:str="float16", # The compute type supported by faster-Whisper
        beam_size=1, # beam size
)->None:
    """A utility function for calculing WER in Common voice dataset provided a model name in huggingface.
       You can store a WER, CER, ModelSize, TimeList to calculate results cumulatively over different epochs
    """
    dataset = load_malayalam_speech_corpus_dataset()
    model = WhisperModel(model_name, device="cuda", compute_type=compute_type)
     
    
    predictions = []
    references = []

    start = time.time()
    for x in tqdm(dataset):
        segments, info = model.transcribe(x["audio"]["array"], beam_size=beam_size)
        predictions.append(normalizer(" ".join([segment.text for segment in segments])))
        references.append(normalizer(x["transcript"]))
              
    end = time.time()
    print(f"Total time taken: {end - start}")
    timelist.append(end - start)
    
    df = pd.DataFrame({"predictions": predictions, "ground_truth": references})
    df["model_name"] = model_name
    df["wer"] = df.apply(lambda row: wer(normalizer(row["ground_truth"]), normalizer(row["predictions"])), axis=1)
    df["cer"] = df.apply(lambda row: cer(normalizer(row["ground_truth"]), normalizer(row["predictions"])), axis=1)
    df["total_time"] = end-start
    
    rwer = wer(references, predictions)
    rwer = round(100 * rwer, 2)
    werlist.append(rwer)
    print(f"The WER of model: {rwer}")

    rcer = cer(references, predictions)
    rcer = round(100 * rcer, 2)
    cerlist.append(rcer)
    print(f"The CER of model: {rcer}")
    
    # print(f"The model size is: {get_model_size(whisper_asr.model)}")
    # modelsizelist.append(get_model_size(whisper_asr.model))
    # df["model_size"] = get_model_size(whisper_asr.model)
    
    save_name = model_name.split("/")
    print(save_name)
    df.to_parquet(f"{save_name[0]}_{save_name[1]}_msc.parquet")
    
    clear_gpu_memory()

In [23]:
#|eval: false
wer_list = []
cer_list = []
model_size_list = []
time_list = []
evaluate_faster_whisper_model_msc("kurianbenoy/vegam-whisper-medium-ml-fp16", wer_list, cer_list, model_size_list, time_list)
wer_list, cer_list, model_size_list, time_list

  0%|          | 0/1541 [00:00<?, ?it/s]

Total time taken: 1052.7706644535065
The WER of model: 11.66
The CER of model: 10.27
['kurianbenoy', 'vegam-whisper-medium-ml-fp16']


([11.66], [10.27], [], [1052.7706644535065])